## Import libraries

In [1]:
import cassandra
from cassandra.cluster import Cluster

## Create a connection to the database

In [2]:
try:
    cluster = Cluster(["127.0.0.1"])
    session = cluster.connect()
except Exception as e:
    print(e)

## Create a keyspace to do the work in and connect to it

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_keyspace
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
    session.set_keyspace("udacity_keyspace")
except Exception as e:
    print(e)

## Music library 

We want to ask 3 questions of the data:
1. Give every album in the music library that was released in a given year  
`select * from music_library WHERE YEAR=1970`
2. Give every album in the music library that was created by a given artist  
`select * from artist_library WHERE artist_name="The Beatles"`
3. Give all the information from the music library about a given album  
`select * from album_library WHERE album_name="Close To You"`  

Since we want to do three different queries, we will need different tables that partition the data differently.

![music_library_cassandra_db](../images/music_library_cassandra_db.png)

### Create the tables and insert data

#### music_library

In [4]:
# Create table
create_music_library = """
CREATE TABLE IF NOT EXISTS music_library (
    year int,
    artist_name text,
    album_name text,
    PRIMARY KEY (year, artist_name)
    );"""
try:
    session.execute(create_music_library)
except Exception as e:
    print(e)
    
# Insert data
insert_music_library = """
INSERT INTO music_library (year, artist_name, album_name)
VALUES (%s, %s, %s);
"""
music_library_data = [
    (1970, "The Beatles", "Let it Be"),
    (1965, "The Beatles", "Rubber Soul"),
    (1965, "The Who", "My Generation"),
    (1966, "The Monkees", "The Monkees"),
    (1970, "The Carpenters", "Close To You")
]
try:
    for item in music_library_data:
        session.execute(insert_music_library, item)
except Exception as e:
    print(e)

#### artist_library

In [5]:
# Create table
create_artist_library = """
CREATE TABLE IF NOT EXISTS artist_library (
    artist_name text,
    album_name text,
    year int,
    PRIMARY KEY (artist_name, album_name)
    );"""
try:
    session.execute(create_artist_library)
except Exception as e:
    print(e)
    
# Insert data
insert_artist_library = """
INSERT INTO artist_library (artist_name, album_name, year)
VALUES (%s, %s, %s);
"""
artist_library_data = [
    ("The Beatles", "Let it Be", 1970),
    ("The Beatles", "Rubber Soul", 1965),
    ("The Who", "My Generation", 1965),
    ("The Monkees", "The Monkees", 1966),
    ("The Carpenters", "Close To You", 1970)
]
try:
    for item in artist_library_data:
        session.execute(insert_artist_library, item)
except Exception as e:
    print(e)

#### album_library

In [6]:
# Create table
create_album_library = """
CREATE TABLE IF NOT EXISTS album_library (
    album_name text,
    artist_name text,
    year int,
    PRIMARY KEY (album_name, artist_name)
    );"""
try:
    session.execute(create_album_library)
except Exception as e:
    print(e)
    
# Insert data
insert_album_library = """
INSERT INTO album_library (album_name, artist_name, year)
VALUES (%s, %s, %s);
"""
album_library_data = [
    ("Let it Be", "The Beatles", 1970),
    ("Rubber Soul", "The Beatles", 1965),
    ("My Generation", "The Who", 1965),
    ("The Monkees", "The Monkees", 1966),
    ("Close To You", "The Carpenters", 1970)
]
try:
    for item in album_library_data:
        session.execute(insert_album_library, item)
except Exception as e:
    print(e)

### Validate the data model

#### Query 1

In [7]:
query = """
SELECT *
FROM music_library
WHERE year = 1970;
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.artist_name, row.album_name)

1970 The Beatles Let it Be
1970 The Carpenters Close To You


#### Query 2

In [8]:
query = """
SELECT *
FROM artist_library
WHERE artist_name = 'The Beatles';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.album_name, row.year)

The Beatles Let it Be 1970
The Beatles Rubber Soul 1965


#### Query 3

In [9]:
query = """
SELECT *
FROM album_library
WHERE album_name = 'Close To You';
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.album_name, row.artist_name, row.year)

Close To You The Carpenters 1970


## Drop the tables

In [10]:
try:
    session.execute("DROP TABLE music_library;")
    session.execute("DROP TABLE artist_library;")
    session.execute("DROP TABLE album_library;")
except Exception as e:
    print(e)

## Close session and cluster connection

In [11]:
session.shutdown()
cluster.shutdown()